In [ ]:
! pip install imdbPY

In [ ]:
import imdb
import csv
import pandas as pd
from imdb import IMDb, IMDbDataAccessError

In [ ]:
ia = imdb.IMDb()
csv_file = '/kaggle/input/imdbdata/imdb_data.csv'

In [ ]:
data=pd.read_csv(r'/kaggle/input/movielens/movies.csv')

In [ ]:
#old verison 
import imdb
import csv
import random
import pandas as pd
import concurrent.futures

# Load the movie data
imdbs = data['imdbId']

# Set up the IMDb API
ia = imdb.IMDb()

# Define the batch size for processing
batch_size = 100

# Define the function to get movie data
def get_movie_data(movie_id):
    try:
        movie = ia.get_movie(movie_id)
        ia.update(movie, 'reviews')
        ia.update(movie, 'cast')
        data = {
            'film_name': movie.get('title', ''),
            'film_rate': movie.get('rating', 0.0),  # default to 0.0 if no rating
            'plot': movie.get('plot', ''),
            'cover_url': movie.get('cover url', ''),
            'cast': ', '.join([person['name'] for person in movie.get('cast', [])[:3]]) if movie.get('cast') else '',  # return empty string if cast is not available
            'reviews': movie.get('reviews', [{}])[0].get('content', '')  # default to empty string if no reviews
        }
        return data
    except IMDbDataAccessError as e:
        return {'film_name': '', 'film_rate': 0.0, 'plot': '', 'cover_url': '', 'cast': '', 'reviews': ''}  # return default values
    except urllib.error.HTTPError as e:
        return {'film_name': '', 'film_rate': 0.0, 'plot': '', 'cover_url': '', 'cast': '', 'reviews': ''}  # return default values

# Define the function to process a batch of movie IDs
def process_batch(movie_ids):
    data_list = []
    for movie_id in movie_ids:
        data_list.append(get_movie_data(movie_id))
    return data_list

# Create a list to store the processed data
data_list = []

# Create a ThreadPoolExecutor to parallelize the API calls
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    # Split the movie IDs into batches
    batches = [imdbs[i:i + batch_size] for i in range(0, len(imdbs), batch_size)]
    
    # Process each batch in parallel
    futures = [executor.submit(process_batch, batch) for batch in batches]
    
    # Collect the results
    for future in concurrent.futures.as_completed(futures):
        data_list.extend(future.result())
csv_file='/kaggle/input/imdbdata/imdb_data.csv'
# Save the processed data to a CSV file
with open(csv_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['film_name', 'film_rate', 'plot', 'cover_url', 'cast', 'reviews'])
    for data in data_list:
        writer.writerow(list(data.values()))

# updated version with more features 

In [ ]:
import imdb
import csv
import random
import pandas as pd
import concurrent.futures
import urllib.error


# Load the movie data
imdbs = data['imdbId']

# Set up the IMDb API
ia = imdb.IMDb()

# Define the batch size for processing
batch_size = 100

# Define the function to get movie data
def get_movie_data(movie_id):
    try:
        movie = ia.get_movie(movie_id)
        ia.update(movie, 'reviews')
        ia.update(movie, 'cast')
        ia.update(movie, 'directors')
        ia.update(movie, 'writers')

        data = {
            'imdbId': movie_id,
            'film_name': movie.get('title', ''),
            'film_rate': movie.get('rating', 0.0),  # default to 0.0 if no rating
            'plot': movie.get('plot', [''])[0],  # take the first plot synopsis
            'cover_url': movie.get('cover url', ''),
            'cast': ', '.join([person['name'] for person in movie.get('cast', [])[:3]]) if movie.get('cast') else '',  # top 3 cast
            'reviews': movie.get('reviews', [{}])[0].get('content', ''),  # first review content
            'director': ', '.join([person['name'] for person in movie.get('directors', [])]) if movie.get('directors') else '',
            'writer': ', '.join([person.get('name', '') for person in movie.get('writers', [])]) if movie.get('writers') else '',
            'genre': ', '.join(movie.get('genres', [])) if movie.get('genres') else ''  # join genre list
        }
        return data
    except imdb.IMDbDataAccessError as e:
        return {'imdbId': movie_id, 'film_name': '', 'film_rate': 0.0, 'plot': '', 'cover_url': '', 'cast': '', 'reviews': '', 'director': '', 'writer': '', 'genre': ''}
    except urllib.error.HTTPError as e:
        return {'imdbId': movie_id, 'film_name': '', 'film_rate': 0.0, 'plot': '', 'cover_url': '', 'cast': '', 'reviews': '', 'director': '', 'writer': '', 'genre': ''}

# Define the function to process a batch of movie IDs
def process_batch(movie_ids):
    data_list = []
    for movie_id in movie_ids:
        data_list.append(get_movie_data(movie_id))
    return data_list

# Create a list to store the processed data
data_list = []

# Create a ThreadPoolExecutor to parallelize the API calls
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    # Split the movie IDs into batches
    batches = [imdbs[i:i + batch_size] for i in range(0, len(imdbs), batch_size)]
    
    # Process each batch in parallel
    futures = [executor.submit(process_batch, batch) for batch in batches]
    
    # Collect the results
    for future in concurrent.futures.as_completed(futures):
        data_list.extend(future.result())

# Specify the path to save the CSV
csv_file = '/kaggle/input/imdbdata/imdb_data.csv'

# Save the processed data to a CSV file
with open(csv_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['imdbId', 'film_name', 'film_rate', 'plot', 'cover_url', 'cast', 'reviews', 'director', 'writer', 'genre'])
    for data in data_list:
        writer.writerow([data['imdbId'], data['film_name'], data['film_rate'], data['plot'], data['cover_url'], data['cast'], data['reviews'], data['director'], data['writer'], data['genre']])


# missing features only 

In [ ]:
import imdb
import csv
import pandas as pd
import concurrent.futures
import urllib.error

imdbs = data['imdbId']

ia = imdb.IMDb()

batch_size = 100

# Define the function to get movie data
def get_movie_data(movie_id):
    try:
        movie = ia.get_movie(movie_id)
        ia.update(movie, info=['directors', 'genres'])  # Only updating required fields

        data = {
            'imdbId': movie_id,
            'film_name': movie.get('title', ''),
            'director': ', '.join([person['name'] for person in movie.get('directors', [])]) if movie.get('directors') else '',
            'genre': ', '.join(movie.get('genres', [])) if movie.get('genres') else ''
        }
        return data
    except imdb.IMDbDataAccessError as e:
        # Handle IMDb Data Access Error
        return {'imdbId': movie_id, 'film_name': '', 'director': '', 'genre': ''}
    except urllib.error.HTTPError as e:
        # Handle HTTP Error
        return {'imdbId': movie_id, 'film_name': '', 'director': '', 'genre': ''}

# Define the function to process a batch of movie IDs
def process_batch(movie_ids):
    data_list = []
    for movie_id in movie_ids:
        data_list.append(get_movie_data(movie_id))
    return data_list

# Create a list to store the processed data
data_list = []

# Create a ThreadPoolExecutor to parallelize the API calls
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    # Split the movie IDs into batches
    batches = [imdbs[i:i + batch_size] for i in range(0, len(imdbs), batch_size)]
    
    # Process each batch in parallel
    futures = [executor.submit(process_batch, batch) for batch in batches]
    
    # Collect the results
    for future in concurrent.futures.as_completed(futures):
        data_list.extend(future.result())

# Specify the path to save the CSV
csv_file = '/kaggle/input/imdbdata/imdb_data.csv'

# Save the processed data to a CSV file with only imdbId, film_name, director, and genre
with open(csv_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['imdbId', 'film_name', 'director', 'genre'])  # Header row
    for data in data_list:
        writer.writerow([data['imdbId'], data['film_name'], data['director'], data['genre']])


In [ ]:
# Specify the path to save the CSV in the writable /kaggle/working directory
csv_file = '/kaggle/working/imdb_data.csv'  # Change the directory to /kaggle/working

# Save the processed data to a CSV file with only imdbId, film_name, director, and genre
with open(csv_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['imdbId', 'film_name', 'director', 'genre'])  # Header row
    for data in data_list:
        writer.writerow([data['imdbId'], data['film_name'], data['director'], data['genre']])


# handling data more effectivly

In [ ]:
import imdb
import csv
import pandas as pd
import concurrent.futures
import urllib.error
import time
from tqdm import tqdm  # For progress tracking

# Load the movie IDs (Assuming 'data' is a DataFrame that contains imdbId)
imdbs = data['imdbId']

# Set up the IMDb API
ia = imdb.IMDb()

# Define the batch size for processing
batch_size = 100

# Define the function to get movie data with retry logic
def get_movie_data(movie_id, retries=3):
    for _ in range(retries):
        try:
            movie = ia.get_movie(movie_id)

            # Fetch the required data without explicitly updating 'directors' or 'genres'
            data = {
                'imdbId': movie_id,
                'film_name': movie.get('title', ''),
                'director': ', '.join([person['name'] for person in movie.get('directors', [])]) if movie.get('directors') else '',
                'genre': ', '.join(movie.get('genres', [])) if movie.get('genres') else ''
            }
            return data
        except (imdb.IMDbDataAccessError, urllib.error.HTTPError) as e:
            time.sleep(1)  # Wait for 1 second before retrying
    # If retries are exhausted, return empty data
    return {'imdbId': movie_id, 'film_name': '', 'director': '', 'genre': ''}

# Define the function to process a batch of movie IDs
def process_batch(movie_ids):
    data_list = []
    for movie_id in movie_ids:
        data_list.append(get_movie_data(movie_id))
    return data_list

# Create a list to store the processed data
data_list = []

# Create a ThreadPoolExecutor to parallelize the API calls
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    # Split the movie IDs into batches
    batches = [imdbs[i:i + batch_size] for i in range(0, len(imdbs), batch_size)]
    
    # Process each batch in parallel and use tqdm for progress tracking
    futures = [executor.submit(process_batch, batch) for batch in batches]
    
    # Collect the results with progress tracking
    for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures), desc="Processing batches"):
        data_list.extend(future.result())

# Specify the path to save the CSV
csv_file = '/kaggle/input/imdbdata/imdb_data.csv'

# Save the processed data to a CSV file with only imdbId, film_name, director, and genre
with open(csv_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['imdbId', 'film_name', 'director', 'genre'])  # Header row
    for data in data_list:
        writer.writerow([data['imdbId'], data['film_name'], data['director'], data['genre']])


# updated needed features to current file

In [ ]:
import imdb
import pandas as pd
import concurrent.futures

# Extract the list of IMDb IDs
imdbs = data['imdbId']

# Set up the IMDb API
ia = imdb.IMDb()

# Define the batch size for processing
batch_size = 100

# Define the function to get movie data
def get_movie_data(movie_id):
    try:
        movie = ia.get_movie(movie_id)
        ia.update(movie, 'reviews')
        ia.update(movie, 'cast')
        ia.update(movie, 'directors')
        ia.update(movie, 'writers')

        # Return the new data, using `.get()` to safely handle missing fields
        data = {
            'imdbId': movie_id,
            'director': ', '.join([person.get('name', '') for person in movie.get('directors', [])]) if movie.get('directors') else '',
            'writer': ', '.join([person.get('name', '') for person in movie.get('writers', [])]) if movie.get('writers') else '',
            'genre': ', '.join(movie.get('genres', [])) if movie.get('genres') else ''  # join genre list
        }
        return data
    except imdb.IMDbDataAccessError as e:
        return {'imdbId': movie_id, 'director': '', 'writer': '', 'genre': ''}
    
# Define the function to process a batch of movie IDs
def process_batch(movie_ids):
    data_list = []
    for movie_id in movie_ids:
        data_list.append(get_movie_data(movie_id))
    return data_list

# Create a list to store the processed data
data_list = []

# Create a ThreadPoolExecutor to parallelize the API calls
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    # Split the movie IDs into batches
    batches = [imdbs[i:i + batch_size] for i in range(0, len(imdbs), batch_size)]
    
    # Process each batch in parallel
    futures = [executor.submit(process_batch, batch) for batch in batches]
    
    # Collect the results
    for future in concurrent.futures.as_completed(futures):
        data_list.extend(future.result())

# Convert the collected data into a DataFrame
new_data_df = pd.DataFrame(data_list)

# Merge the new data with the existing dataset
updated_data = pd.merge(data, new_data_df, on='imdbId', how='left')

# Save the updated data back to the CSV file
updated_data.to_csv(csv_file, index=False)


In [ ]:
data_list

# download the csv file

In [ ]:
import pandas as pd

# Create a Pandas DataFrame from the data_list
df = pd.DataFrame(data_list)

df.to_csv('imdb_data.csv', index=False)
from IPython.display import FileLink

# This will create a download link for the file
FileLink('imdb_data.csv')

# assosication

In [ ]:
import pandas as pd
from mlxtend.frequent_patterns import fpgrowth, association_rules

# Load the dataset in chunks
chunksize = 1000
chunks = pd.read_csv(r'/kaggle/input/rating/ratings.csv', chunksize=chunksize)

# Concatenate all chunks into a single DataFrame
ratings = pd.concat(chunks, ignore_index=True)

# Filter only ratings of 4.0 and above for "liked" movies
ratings_filtered = ratings[ratings['rating'] >= 4.0]

# Create a user-item matrix
user_movie_matrix = ratings_filtered.pivot_table(index='userId', columns='movieId', aggfunc='size', fill_value=0)

# Apply FP-Growth to find frequent item sets
frequent_itemsets = fpgrowth(user_movie_matrix, min_support=0.02, use_colnames=True)

# Generate association rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)

# Sort the rules by lift
rules = rules.sort_values(by='lift', ascending=False)

# Display the rules
print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']])


# assosiation model 

In [ ]:
import pandas as pd
from mlxtend.frequent_patterns import fpgrowth, association_rules

# Load the dataset in chunks
chunksize = 1000
chunks = pd.read_csv(r'/kaggle/input/rating/ratings.csv', chunksize=chunksize)

# Concatenate all chunks into a single DataFrame
ratings = pd.concat(chunks, ignore_index=True)

# Filter only ratings of 4.0 and above for "liked" movies
ratings_filtered = ratings[ratings['rating'] >= 4.0]

# Create a user-item matrix, where 1 represents a "liked" movie (rating 4.0 or above)
user_movie_matrix = ratings_filtered.pivot_table(index='userId', columns='movieId', aggfunc='size', fill_value=0)

# Apply FP-Growth to find frequent item sets
frequent_itemsets = fpgrowth(user_movie_matrix, min_support=0.02, use_colnames=True)

# Generate association rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)

# Sort the rules by lift
rules = rules.sort_values(by='lift', ascending=False)

# Load the movies dataset to map movie IDs to movie titles
movies = pd.read_csv(r'/kaggle/input/movielens/movies.csv')

# Create a function to get movie recommendations
def get_movie_id(movie_title):
    """
    Get the movieId for the given movie title from the movies dataset.
    """
    movie_id = movies[movies['title'].str.contains(movie_title, case=False, na=False)]['movieId']
    if len(movie_id) > 0:
        return movie_id.values[0]
    else:
        return None

def recommend_movies(movie_title, rules, movies, top_n=3):
    """
    Recommend associated movies based on the input movie title.
    """
    movie_id = get_movie_id(movie_title)
    if movie_id is None:
        return f"Movie '{movie_title}' not found in the dataset."

    # Find rules where the input movie is in the antecedents
    matching_rules = rules[rules['antecedents'].apply(lambda x: movie_id in x)]

    if matching_rules.empty:
        return f"No associated movies found for '{movie_title}'."

    # Extract the consequents from the association rules
    movie_recommendations = []
    for _, row in matching_rules.iterrows():
        for consequent in row['consequents']:
            if consequent != movie_id:
                movie_recommendations.append((consequent, row['lift']))

    # Remove duplicates and sort by lift value (descending)
    movie_recommendations = sorted(list(set(movie_recommendations)), key=lambda x: x[1], reverse=True)

    # Get the top N movie recommendations
    recommended_movie_ids = [rec[0] for rec in movie_recommendations[:top_n]]
    recommended_movies = movies[movies['movieId'].isin(recommended_movie_ids)]['title'].values

    return recommended_movies

# Example usage:
movie_name = 'Interstellar'  # Change this to the movie title you want recommendations for
recommended_movies = recommend_movies(movie_name, rules, movies)

if isinstance(recommended_movies, str):
    print(recommended_movies)  # If the function returns an error message
else:
    print(f"Movies recommended based on '{movie_name}':")
    for i, movie in enumerate(recommended_movies, 1):
        print(f"{i}. {movie}")


# assosiation model low resources

In [ ]:
import pandas as pd
from mlxtend.frequent_patterns import fpgrowth, association_rules
from scipy.sparse import csr_matrix

# Load the dataset in chunks to avoid memory overload
chunksize = 10**6  # Increase the chunksize to process more data at once but keep it manageable
chunks = pd.read_csv(r'/kaggle/input/rating/ratings.csv', chunksize=chunksize, usecols=['userId', 'movieId', 'rating'])

# Initialize an empty list to store filtered data
filtered_chunks = []

# Process each chunk individually
for chunk in chunks:
    # Filter ratings of 4.0 and above for "liked" movies
    filtered_chunk = chunk[chunk['rating'] >= 4.0]
    filtered_chunks.append(filtered_chunk)

# Concatenate all filtered chunks into a single DataFrame
ratings_filtered = pd.concat(filtered_chunks, ignore_index=True)

# Create a user-item matrix in sparse format to save memory
user_movie_matrix = ratings_filtered.pivot_table(index='userId', columns='movieId', aggfunc='size', fill_value=0)
user_movie_matrix_sparse = csr_matrix(user_movie_matrix.values)

# Apply FP-Growth to find frequent item sets
frequent_itemsets = fpgrowth(pd.DataFrame(user_movie_matrix_sparse.todense(), columns=user_movie_matrix.columns), min_support=0.02, use_colnames=True)

# Generate association rules from the frequent item sets
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)

# Sort the rules by lift to find the most interesting ones
rules = rules.sort_values(by='lift', ascending=False)

# Load the movies dataset to map movie IDs to movie titles
movies = pd.read_csv(r'/kaggle/input/movielens/movies.csv')

# Create a function to get movie recommendations
def get_movie_id(movie_title):
    """
    Get the movieId for the given movie title from the movies dataset.
    """
    movie_id = movies[movies['title'].str.contains(movie_title, case=False, na=False)]['movieId']
    if len(movie_id) > 0:
        return movie_id.values[0]
    else:
        return None

def recommend_movies(movie_title, rules, movies, top_n=3):
    """
    Recommend associated movies based on the input movie title.
    """
    movie_id = get_movie_id(movie_title)
    if movie_id is None:
        return f"Movie '{movie_title}' not found in the dataset."

    # Find rules where the input movie is in the antecedents
    matching_rules = rules[rules['antecedents'].apply(lambda x: movie_id in x)]

    if matching_rules.empty:
        return f"No associated movies found for '{movie_title}'."

    # Extract the consequents from the association rules
    movie_recommendations = []
    for _, row in matching_rules.iterrows():
        for consequent in row['consequents']:
            if consequent != movie_id:
                movie_recommendations.append((consequent, row['lift']))

    # Remove duplicates and sort by lift value (descending)
    movie_recommendations = sorted(list(set(movie_recommendations)), key=lambda x: x[1], reverse=True)

    # Get the top N movie recommendations
    recommended_movie_ids = [rec[0] for rec in movie_recommendations[:top_n]]
    recommended_movies = movies[movies['movieId'].isin(recommended_movie_ids)]['title'].values

    return recommended_movies

# Example usage:
movie_name = 'Interstellar'  # Change this to the movie title you want recommendations for
recommended_movies = recommend_movies(movie_name, rules, movies)

if isinstance(recommended_movies, str):
    print(recommended_movies)  # If the function returns an error message
else:
    print(f"Movies recommended based on '{movie_name}':")
    for i, movie in enumerate(recommended_movies, 1):
        print(f"{i}. {movie}")


# final try 

In [ ]:
import pandas as pd
from mlxtend.frequent_patterns import fpgrowth, association_rules
from scipy.sparse import csr_matrix
import re

# Function to clean movie titles by removing special characters and lowercasing
def clean_movie_title(title):
    # Remove anything in parentheses (like year)
    title = re.sub(r'\(.*?\)', '', title)
    # Remove extra spaces and lowercase the title
    title = re.sub(r'\s+', ' ', title).strip().lower()
    return title

# Load the dataset in chunks
chunksize = 10**6
chunks = pd.read_csv(r'/kaggle/input/rating/ratings.csv', chunksize=chunksize, usecols=['userId', 'movieId', 'rating'])

# Initialize an empty list to store filtered data
filtered_chunks = []

# Process each chunk individually
for chunk in chunks:
    # Filter ratings of 4.0 and above for "liked" movies
    filtered_chunk = chunk[chunk['rating'] >= 4.0]
    filtered_chunks.append(filtered_chunk)

# Concatenate filtered chunks
ratings_filtered = pd.concat(filtered_chunks, ignore_index=True)

# Create a user-item matrix in sparse format
user_movie_matrix = ratings_filtered.pivot_table(index='userId', columns='movieId', aggfunc='size', fill_value=0)
user_movie_matrix_sparse = csr_matrix(user_movie_matrix.values)

# Apply FP-Growth to find frequent item sets
frequent_itemsets = fpgrowth(pd.DataFrame(user_movie_matrix_sparse.todense(), columns=user_movie_matrix.columns), min_support=0.05, use_colnames=True)

# Generate association rules from frequent item sets
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)

# Sort rules by lift
rules = rules.sort_values(by='lift', ascending=False)

# Load the movies dataset
movies = pd.read_csv(r'/kaggle/input/movielens/movies.csv')

# Apply the clean_movie_title function to standardize movie titles
movies['clean_title'] = movies['title'].apply(clean_movie_title)

# Create a function to get movie recommendations
def get_movie_id(movie_title):
    """
    Get the movieId for the given movie title from the movies dataset.
    Clean the input movie title for consistency.
    """
    cleaned_title = clean_movie_title(movie_title)
    movie_id = movies[movies['clean_title'] == cleaned_title]['movieId']
    
    if len(movie_id) > 0:
        return movie_id.values[0]
    else:
        return None

def recommend_movies(movie_title, rules, movies, top_n=3):
    """
    Recommend associated movies based on the input movie title.
    """
    movie_id = get_movie_id(movie_title)
    if movie_id is None:
        return f"Movie '{movie_title}' not found in the dataset."

    # Find rules where the input movie is in the antecedents
    matching_rules = rules[rules['antecedents'].apply(lambda x: movie_id in x)]

    if matching_rules.empty:
        return f"No associated movies found for '{movie_title}'."

    # Extract the consequents from the association rules
    movie_recommendations = []
    for _, row in matching_rules.iterrows():
        for consequent in row['consequents']:
            if consequent != movie_id:
                movie_recommendations.append((consequent, row['lift']))

    # Remove duplicates and sort by lift value
    movie_recommendations = sorted(list(set(movie_recommendations)), key=lambda x: x[1], reverse=True)

    # Get top N movie recommendations
    recommended_movie_ids = [rec[0] for rec in movie_recommendations[:top_n]]
    recommended_movies = movies[movies['movieId'].isin(recommended_movie_ids)]['title'].values

    return recommended_movies



In [ ]:
# Example usage:
movie_name = 'Heat (1995)'  # You can try different variations like 'interstellar'
recommended_movies = recommend_movies(movie_name, rules, movies)

if isinstance(recommended_movies, str):
    print(recommended_movies)  # Error message
else:
    print(f"Movies recommended based on '{movie_name}':")
    for i, movie in enumerate(recommended_movies, 1):
        print(f"{i}. {movie}")

# cosine matrix improved

In [ ]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load the movie dataset
data = pd.read_csv(r'D:\abdo\AI\projects\recommender\needed features\imdb_data.csv')

# Convert data to a DataFrame
df = pd.DataFrame(data)

# Replace NaN values in relevant columns with an empty string
df['overview'] = df['overview'].fillna('')
df['reviews'] = df['reviews'].fillna('')
df['cast'] = df['cast'].fillna('')
df['director'] = df['director'].fillna('')
df['genre'] = df['genre'].fillna('')

# Combine 'title', 'overview', 'cast', 'director', 'genre', and 'reviews' for similarity comparison
df['combined_features'] = df['title'] + ' ' + df['overview'] + ' ' + df['cast'] + ' ' + df['director'] + ' ' + df['genre'] + ' ' + df['reviews']

# Text Vectorization using TF-IDF (including bi-grams for better context)
tfidf = TfidfVectorizer(stop_words='english', ngram_range=(1, 2))
tfidf_matrix = tfidf.fit_transform(df['combined_features'].fillna(''))

# Calculate cosine similarity between all movies
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Helper function to extract movie title
def extract_title(title):
    if pd.isna(title):
        return ''
    title = title.lower()
    title = re.sub(r'\s*\(\d{4}\)$', '', title)  # Remove year information
    return title

# Define the function to recommend movies
def recommend_movies(movie_title, cosine_sim=cosine_sim, df=df):
    movie_title_clean = extract_title(movie_title)
    
    # Check if the movie title exists in the dataset
    if movie_title_clean in df['title'].apply(extract_title).values:
        # Get the index of the movie that matches the title
        idx = df.index[df['title'].apply(extract_title) == movie_title_clean][0]
        
        # Get the pairwise similarity scores of all movies with that movie
        sim_scores = list(enumerate(cosine_sim[idx]))
        
        # Sort the movies based on similarity scores
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        
        # Get the top 3 most similar movies (excluding the input movie itself)
        recommended_movies = []
        for score in sim_scores[1:]:
            movie_data = df.iloc[score[0]]
            recommended_movies.append({
                'title': movie_data['title'],
                'film_rate': movie_data['film_rate'],
                'imdbId': movie_data['imdbId'],
                'cover_url': movie_data['cover_url'],
                'director': movie_data['director'],
                'genre': movie_data['genre'],
                'cast': movie_data['cast'],
                'overview': movie_data['overview'],
                'reviews': movie_data['reviews']
            })
            if len(recommended_movies) == 3:  # Limit to top 3 movies
                break
        
        # Return the top 3 most similar movies with all requested details
        return recommended_movies
    else:
        return []  # Return an empty list if the movie is not found

# Example usage: Recommend movies based on the title "Heat"
test = 'heat'
recommended_movies = recommend_movies(test)
print(f"Movies recommended based on '{test}':")
for movie in recommended_movies:
    print(f"Title: {movie['title']}, Rating: {movie['film_rate']}, IMDb ID: {movie['imdbId']}")
    print(f"Director: {movie['director']}, Genre: {movie['genre']}")
    print(f"Cast: {movie['cast']}")
    print(f"Overview: {movie['overview']}")
    print(f"Reviews: {movie['reviews']}")
    print(f"Cover URL: {movie['cover_url']}")
    print("-" * 80)


# LSTM

In [2]:
pip install tensorflow

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Note: you may need to restart the kernel to use updated packages.


# lstm 256 patch size adam w 

In [13]:
import pandas as pd
import re
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.metrics.pairwise import cosine_similarity

# Load the movie dataset
data = pd.read_csv(r'/kaggle/input/movies-updated/movies_with_tags_rating.csv')

# Preprocessing
df = pd.DataFrame(data)
df['overview'] = df['overview'].fillna('')
df['cast'] = df['cast'].fillna('')
df['director'] = df['director'].fillna('(no data about the director is found)')
df['genre'] = df['genre'].fillna('')
df['tag'] = df['tag'].fillna('')
df['reviews'] = df['reviews'].fillna('')

# Combine features into a single column for text processing
df['combined_features'] = df['overview'] + ' ' + df['cast'] + ' ' + df['director'] + ' ' + df['genre'] + ' ' + df['tag'] + ' ' + df['reviews']

# Tokenization and padding
max_words = 10000
max_sequence_len = 300

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(df['combined_features'])
sequences = tokenizer.texts_to_sequences(df['combined_features'])
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_len, padding='post', truncating='post')

# Define LSTM model
embedding_dim = 100

model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=max_sequence_len))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='AdamW', metrics=['accuracy'])
model.summary()

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, df['rating'], test_size=0.2, random_state=42)

# Train the LSTM model
model.fit(X_train, y_train, epochs=5, batch_size=256, validation_data=(X_test, y_test))

# Generate embeddings from the trained LSTM model
embedding_layer = Sequential()
embedding_layer.add(model.layers[0])
embedding_layer.add(model.layers[1])
embedding_layer.compile(loss='binary_crossentropy', optimizer='AdamW')

# Generate embeddings for all movies
movie_embeddings = embedding_layer.predict(padded_sequences)

# Flatten the LSTM output by averaging over timesteps
movie_embeddings_2d = np.mean(movie_embeddings, axis=1)

# Calculate cosine similarity based on 2D embeddings
cosine_sim_lstm = cosine_similarity(movie_embeddings_2d, movie_embeddings_2d)

# Function to clean movie titles (remove special characters and lowercase)
def extract_title(title):
    # Check if the title is NaN or not a string, return an empty string in that case
    if isinstance(title, str):
        title_cleaned = title.lower()
        title_cleaned = re.sub(r'\W+', ' ', title_cleaned)
        return title_cleaned.strip()
    else:
        return np.nan

# Function to recommend movies based on LSTM embeddings
def recommend_movies_lstm(movie_title, cosine_sim=cosine_sim_lstm, df=df):
    movie_title_clean = extract_title(movie_title)
    
    # Check if the movie title exists in the dataset
    if movie_title_clean in df['title'].apply(extract_title).values:
        # Get the index of the movie that matches the title
        idx = df.index[df['title'].apply(extract_title) == movie_title_clean][0]
        
        # Get the pairwise similarity scores of all movies with that movie
        sim_scores = list(enumerate(cosine_sim[idx]))
        
        # Sort the movies based on similarity scores
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        
        # Get the top 3 most similar movies (excluding the input movie itself)
        recommended_movies = []
        for score in sim_scores[1:]:
            movie_data = df.iloc[score[0]]
            recommended_movies.append({
                'title': movie_data['title'],
                'film_rate': movie_data['film_rate'],
                'imdbId': movie_data['imdbId'],
                'cover_url': movie_data['cover_url'],
                'director': movie_data['director'],
                'genre': movie_data['genre'],
                'cast': movie_data['cast'],
                'overview': movie_data['overview'],
                'reviews': movie_data['reviews'],
                'tag': movie_data['tag'],
                'rating': movie_data['rating']
            })
            if len(recommended_movies) == 3:  # Limit to top 3 movies
                break
        
        # Return the top 3 most similar movies with all requested details
        return recommended_movies
    else:
        return []  # Return an empty list if the movie is not found
# Example usage with LSTM model
test_movie = 'heat'
recommended_movies_lstm = recommend_movies_lstm(test_movie)

if not recommended_movies_lstm:
    print(f"'{test_movie}' is not listed in our database.")
else:
    print(f"Movies recommended based on '{test_movie}':")
    for movie in recommended_movies_lstm:
        print(f"Title: {movie['title']}, Rating: {movie['film_rate']}, User Rating: {movie['rating']}, Tags: {movie['tag']}")
        print(f"Director: {movie['director']}")
        print(f"Genre: {movie['genre']}")
        print("-" * 80)


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_7 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_14 (LSTM)                  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_15 (LSTM)                  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
33/33 ━━━━━━━━━━━━━━━━━━━━ 5s 75ms/step - accuracy: 5.9395e-04 - loss: -2.3343 - val_accuracy: 0.0024 - val_loss: -14.6580
Epoch 2/5
33/33 ━━━━━━━━━━━━━━━━━━━━ 2s 65ms/step - accuracy: 5.0738e-04 - loss: -19.2410 - val_accuracy: 0.0024 - val_loss: -35.8940
Epoch 3/5
33/33 ━━━━━━━━━━━━━━━━━━━━ 2s 66ms/step - accuracy: 8.8278e-04 - loss: -44.2356 - val_accuracy: 0.0024 - val_loss: -74.9139
Epoch 4/5
33/33 ━━━━━━━━━━━━━━━━━━━━ 2s 65ms/step - accuracy: 4.3675e-04 - loss: -89.7688 - val_accuracy: 0.0024 - val_loss: -141.4596
Epoch 5/5
33/33 ━━━━━━━━━━━━━━━━━━━━ 2s 65ms/step - accuracy: 2.9908e-04 - loss: -165.9277 - val_accuracy: 0.0024 - val_loss: -245.6596
330/330 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step
Movies recommended based on 'heat':
Title: Hope Springs, Rating: 6.3, User Rating: 3.15, Tags: 
Director: David Frankel
Genre: Comedy, Drama, Romance
--------------------------------------------------------------------------------
Title: The Prophecy, Rating: 6.4, User Rating: 3.2, Tags

# lstm 256 batch adam

In [ ]:
import pandas as pd
import re
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.metrics.pairwise import cosine_similarity

# Load the movie dataset
data = pd.read_csv(r'/kaggle/input/movies-updated/movies_with_tags_rating.csv')

# Preprocessing
df = pd.DataFrame(data)
df['overview'] = df['overview'].fillna('')
df['cast'] = df['cast'].fillna('')
df['director'] = df['director'].fillna('(no data about the director is found)')
df['genre'] = df['genre'].fillna('')
df['tag'] = df['tag'].fillna('')
df['reviews'] = df['reviews'].fillna('')

# Combine features into a single column for text processing
df['combined_features'] = df['overview'] + ' ' + df['cast'] + ' ' + df['director'] + ' ' + df['genre'] + ' ' + df['tag'] + ' ' + df['reviews']

# Tokenization and padding
max_words = 10000
max_sequence_len = 300

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(df['combined_features'])
sequences = tokenizer.texts_to_sequences(df['combined_features'])
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_len, padding='post', truncating='post')

# Define LSTM model
embedding_dim = 100

model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=max_sequence_len))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, df['rating'], test_size=0.2, random_state=42)

# Train the LSTM model
model.fit(X_train, y_train, epochs=5, batch_size=256, validation_data=(X_test, y_test))

# Generate embeddings from the trained LSTM model
embedding_layer = Sequential()
embedding_layer.add(model.layers[0])
embedding_layer.add(model.layers[1])
embedding_layer.compile(loss='binary_crossentropy', optimizer='Adam')

# Generate embeddings for all movies
movie_embeddings = embedding_layer.predict(padded_sequences)

# Flatten the LSTM output by averaging over timesteps
movie_embeddings_2d = np.mean(movie_embeddings, axis=1)

# Calculate cosine similarity based on 2D embeddings
cosine_sim_lstm = cosine_similarity(movie_embeddings_2d, movie_embeddings_2d)

# Function to clean movie titles (remove special characters and lowercase)
def extract_title(title):
    # Check if the title is NaN or not a string, return an empty string in that case
    if isinstance(title, str):
        title_cleaned = title.lower()
        title_cleaned = re.sub(r'\W+', ' ', title_cleaned)
        return title_cleaned.strip()
    else:
        return np.nan

# Function to recommend movies based on LSTM embeddings
def recommend_movies_lstm(movie_title, cosine_sim=cosine_sim_lstm, df=df):
    movie_title_clean = extract_title(movie_title)
    
    # Check if the movie title exists in the dataset
    if movie_title_clean in df['title'].apply(extract_title).values:
        # Get the index of the movie that matches the title
        idx = df.index[df['title'].apply(extract_title) == movie_title_clean][0]
        
        # Get the pairwise similarity scores of all movies with that movie
        sim_scores = list(enumerate(cosine_sim[idx]))
        
        # Sort the movies based on similarity scores
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        
        # Get the top 3 most similar movies (excluding the input movie itself)
        recommended_movies = []
        for score in sim_scores[1:]:
            movie_data = df.iloc[score[0]]
            recommended_movies.append({
                'title': movie_data['title'],
                'film_rate': movie_data['film_rate'],
                'imdbId': movie_data['imdbId'],
                'cover_url': movie_data['cover_url'],
                'director': movie_data['director'],
                'genre': movie_data['genre'],
                'cast': movie_data['cast'],
                'overview': movie_data['overview'],
                'reviews': movie_data['reviews'],
                'tag': movie_data['tag'],
                'rating': movie_data['rating']
            })
            if len(recommended_movies) == 3:  # Limit to top 3 movies
                break
        
        # Return the top 3 most similar movies with all requested details
        return recommended_movies
    else:
        return []  # Return an empty list if the movie is not found
# Example usage with LSTM model
test_movie = 'heat'
recommended_movies_lstm = recommend_movies_lstm(test_movie)

if not recommended_movies_lstm:
    print(f"'{test_movie}' is not listed in our database.")
else:
    print(f"Movies recommended based on '{test_movie}':")
    for movie in recommended_movies_lstm:
        print(f"Title: {movie['title']}, Rating: {movie['film_rate']}, User Rating: {movie['rating']}, Tags: {movie['tag']}")
        print(f"Director: {movie['director']}")
        print(f"Genre: {movie['genre']}")
        print("-" * 80)


# another try 

In [17]:
import pandas as pd
import re
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.metrics.pairwise import cosine_similarity

# Load the movie dataset
data = pd.read_csv(r'/kaggle/input/movies-updated/movies_with_tags_rating.csv')

# Preprocessing
df = pd.DataFrame(data)
df['overview'] = df['overview'].fillna('')
df['cast'] = df['cast'].fillna('')
df['director'] = df['director'].fillna('(no data about the director is found)')
df['genre'] = df['genre'].fillna('')
df['tag'] = df['tag'].fillna('')
df['reviews'] = df['reviews'].fillna('')

# Feature Weighting
df['combined_features'] = (df['overview'] * 1 + ' ' +
                           df['cast'] * 3 + ' ' + 
                           df['director'] * 4 + ' ' + 
                           df['genre'] * 5 + ' ' + 
                           df['tag'] * 2 + ' ' + 
                           df['reviews'] * 1)
# Tokenization and padding
max_words = 10000
max_sequence_len = 300

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(df['combined_features'])
sequences = tokenizer.texts_to_sequences(df['combined_features'])
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_len, padding='post', truncating='post')

# Define LSTM model
embedding_dim = 100

model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=max_sequence_len))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, df['rating'], test_size=0.2, random_state=42)

# Train the LSTM model
model.fit(X_train, y_train, epochs=5, batch_size=256, validation_data=(X_test, y_test))

# Generate embeddings from the trained LSTM model
embedding_layer = Sequential()
embedding_layer.add(model.layers[0])
embedding_layer.add(model.layers[1])
embedding_layer.compile(loss='binary_crossentropy', optimizer='Adam')

# Generate embeddings for all movies
movie_embeddings = embedding_layer.predict(padded_sequences)

# Flatten the LSTM output by averaging over timesteps
movie_embeddings_2d = np.mean(movie_embeddings, axis=1)

# Calculate cosine similarity based on 2D embeddings
cosine_sim_lstm = cosine_similarity(movie_embeddings_2d, movie_embeddings_2d)

# Function to clean movie titles (remove special characters and lowercase)
def extract_title(title):
    if isinstance(title, str):
        title_cleaned = title.lower()
        title_cleaned = re.sub(r'\W+', ' ', title_cleaned)
        return title_cleaned.strip()
    else:
        return np.nan

# Function to recommend movies based on LSTM embeddings
def recommend_movies_lstm(movie_title, cosine_sim=cosine_sim_lstm, df=df):
    movie_title_clean = extract_title(movie_title)
    
    # Check if the movie title exists in the dataset
    if movie_title_clean in df['title'].apply(extract_title).values:
        # Get the index of the movie that matches the title
        idx = df.index[df['title'].apply(extract_title) == movie_title_clean][0]
        
        # Get the pairwise similarity scores of all movies with that movie
        sim_scores = list(enumerate(cosine_sim[idx]))
        
        # Sort the movies based on similarity scores
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        
        # Get the top 3 most similar movies (excluding the input movie itself)
        recommended_movies = []
        for score in sim_scores[1:]:
            movie_data = df.iloc[score[0]]
            recommended_movies.append({
                'title': movie_data['title'],
                'film_rate': movie_data['film_rate'],
                'imdbId': movie_data['imdbId'],
                'cover_url': movie_data['cover_url'],
                'director': movie_data['director'],
                'genre': movie_data['genre'],
                'cast': movie_data['cast'],
                'overview': movie_data['overview'],
                'reviews': movie_data['reviews'],
                'tag': movie_data['tag'],
                'rating': movie_data['rating']
            })
            if len(recommended_movies) == 3:  # Limit to top 3 movies
                break
        
        # Return the top 3 most similar movies with all requested details
        return recommended_movies
    else:
        return []  # Return an empty list if the movie is not found

# Example usage with LSTM model
test_movie = 'heat'
recommended_movies_lstm = recommend_movies_lstm(test_movie)

if not recommended_movies_lstm:
    print(f"'{test_movie}' is not listed in our database.")
else:
    print(f"Movies recommended based on '{test_movie}':")
    for movie in recommended_movies_lstm:
        print(f"Title: {movie['title']}, Rating: {movie['film_rate']}, User Rating: {movie['rating']}, Tags: {movie['tag']}")
        print(f"Director: {movie['director']}")
        print(f"Genre: {movie['genre']}")
        print("-" * 80)


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_18"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_9 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_18 (LSTM)                  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_19 (LSTM)                  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_27 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_28 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_29 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
33/33 ━━━━━━━━━━━━━━━━━━━━ 5s 75ms/step - accuracy: 6.4984e-04 - loss: -2.4309 - val_accuracy: 0.0024 - val_loss: -16.3625
Epoch 2/5
33/33 ━━━━━━━━━━━━━━━━━━━━ 2s 65ms/step - accuracy: 0.0011 - loss: -22.1989 - val_accuracy: 0.0024 - val_loss: -43.5638
Epoch 3/5
33/33 ━━━━━━━━━━━━━━━━━━━━ 2s 65ms/step - accuracy: 3.4469e-04 - loss: -54.4752 - val_accuracy: 0.0024 - val_loss: -92.9786
Epoch 4/5
33/33 ━━━━━━━━━━━━━━━━━━━━ 2s 65ms/step - accuracy: 9.0351e-04 - loss: -112.7620 - val_accuracy: 0.0024 - val_loss: -181.9569
Epoch 5/5
33/33 ━━━━━━━━━━━━━━━━━━━━ 2s 65ms/step - accuracy: 4.4829e-04 - loss: -215.2816 - val_accuracy: 0.0024 - val_loss: -328.8805
330/330 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step
Movies recommended based on 'heat':
Title: Little Women, Rating: 7.3, User Rating: 3.65, Tags: 
Director: George Cukor
Genre: Drama, Family, Romance, War
--------------------------------------------------------------------------------
Title: Little Women, Rating: 7.3, User Rating: 3.65, Ta

In [18]:
test_movie = 'interstellar'
recommended_movies_lstm = recommend_movies_lstm(test_movie)

if not recommended_movies_lstm:
    print(f"'{test_movie}' is not listed in our database.")
else:
    print(f"Movies recommended based on '{test_movie}':")
    for movie in recommended_movies_lstm:
        print(f"Title: {movie['title']}, Rating: {movie['film_rate']}, User Rating: {movie['rating']}, Tags: {movie['tag']}")
        print(f"Director: {movie['director']}")
        print(f"Genre: {movie['genre']}")
        print("-" * 80)

Movies recommended based on 'interstellar':
Title: Material Girls, Rating: 3.9, User Rating: 1.95, Tags: 
Director: Martha Coolidge
Genre: Comedy, Family, Romance
--------------------------------------------------------------------------------
Title: Babe, Rating: 6.9, User Rating: 3.45, Tags: 
Director: Chris Noonan
Genre: Comedy, Drama, Family
--------------------------------------------------------------------------------
Title: How to Make an American Quilt, Rating: 6.3, User Rating: 3.15, Tags: 
Director: Jocelyn Moorhouse
Genre: Comedy, Drama, Romance
--------------------------------------------------------------------------------
